In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sqlite3
import sys


In [4]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data_raw = pd.read_sql_query(query, conn)
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299670 entries, 0 to 299669
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   uid                        299670 non-null  object 
 1   id                         299670 non-null  object 
 2   nature                     299670 non-null  object 
 3   acheteur_id                299670 non-null  object 
 4   acheteur_nom               299477 non-null  object 
 5   acheteur_siren             299500 non-null  object 
 6   titulaire_id               299670 non-null  object 
 7   titulaire_typeIdentifiant  299670 non-null  object 
 8   titulaire_nom              292097 non-null  object 
 9   titulaire_siren            298537 non-null  object 
 10  objet                      299670 non-null  object 
 11  montant                    299670 non-null  float64
 12  codeCPV                    299670 non-null  object 
 13  procedure                  29

In [5]:
data_raw['typeGroupementOperateurs']

0         Pas de groupement
1         Pas de groupement
2         Pas de groupement
3                 Solidaire
4         Pas de groupement
                ...        
299665    Pas de groupement
299666    Pas de groupement
299667    Pas de groupement
299668    Pas de groupement
299669    Pas de groupement
Name: typeGroupementOperateurs, Length: 299670, dtype: object

In [6]:
X = data_raw[['procedure', 'dureeMois', 'formePrix', 'attributionAvance', 'offresRecues', 'ccag',
          'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance', 'origineUE',
          'origineFrance']]

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299670 entries, 0 to 299669
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   procedure                 299670 non-null  object 
 1   dureeMois                 299663 non-null  float64
 2   nature                    299670 non-null  object 
 3   formePrix                 299670 non-null  object 
 4   attributionAvance         125791 non-null  float64
 5   offresRecues              198227 non-null  float64
 6   ccag                      299670 non-null  object 
 7   sousTraitanceDeclaree     155111 non-null  float64
 8   typeGroupementOperateurs  299670 non-null  object 
 9   tauxAvance                170113 non-null  float64
 10  origineUE                 169189 non-null  float64
 11  origineFrance             169190 non-null  float64
dtypes: float64(7), object(5)
memory usage: 27.4+ MB


In [15]:
X['attributionAvance'].value_counts()

attributionAvance
0.0    107423
1.0     18368
Name: count, dtype: int64

In [17]:
X['dureeMois'].dropna(inplace=True)

/tmp/ipykernel_116173/3308956910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['dureeMois'].dropna(inplace=True)


In [ ]:
X[X['dureeMois'].isna()]

,procedure,dureeMois,nature,formePrix,attributionAvance,offresRecues,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance,origineUE,origineFrance
17607,Procédure adaptée,NaN,Marché,Mixte,1.0,2.0,Travaux,1.0,Pas de groupement,0.05,0.0,0.0
19849,Procédure adaptée,NaN,Marché,Forfaitaire,0.0,4.0,Travaux,1.0,Pas de groupement,0.00,0.0,0.0
42490,Procédure adaptée,NaN,Marché,Unitaire,0.0,4.0,Fournitures courantes et services,0.0,Pas de groupement,0.00,1.0,1.0
168850,Appel d'offres ouvert,NaN,Marché,Forfaitaire,0.0,2.0,Fournitures courantes et services,0.0,Pas de groupement,0.00,0.0,0.0
177724,Appel d'offres ouvert,NaN,Marché,Unitaire,0.0,3.0,Fournitures courantes et services,0.0,Pas de groupement,0.00,0.0,0.0
225857,Appel d'offres ouvert,NaN,Marché,Unitaire,1.0,3.0,Pas de CCAG,0.0,Pas de groupement,0.10,1.0,1.0
286691,Procédure adaptée,NaN,Marché,Forfaitaire,1.0,4.0,Pas de CCAG,0.0,Pas de groupement,0.10,0.0,0.0


In [18]:
X_2 = X.astype({
    'dureeMois': 'int16',
    'attributionAvance': 'int16'
})

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer: Error while type casting for column 'dureeMois'

In [ ]:
from sklearn.ensemble import IsolationForest